In [1]:
# references:
# https://github.com/fpretto/interpretable_and_fair_ml/blob/master/Model%20Interpretation%20and%20Fairness.ipynb
# https://towardsdatascience.com/compas-case-study-fairness-of-a-machine-learning-model-f0f804108751
# !pip install https://github.com/adebayoj/fairml/archive/master.zip

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import xgboost as xgb
from fairml import audit_model
from fairml import plot_dependencies
from sklearn.metrics import confusion_matrix
%matplotlib inline

# compas model

In [6]:
import pandas as pd
df = pd.read_csv('data/compas-scores-two-years.csv')
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
print(df.shape)

(6150, 53)


In [7]:
# score for Black defendants
threshold  = 6
df_black = df[df['race']=="African-American"].copy()
df_black['is_med_or_high_risk'] = (df_black['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_black['two_year_recid'], df_black['is_med_or_high_risk'])
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))

False positive rate (Black)      :  0.34317548746518106
False negative rate (Black)      :  0.37243556023145713


In [8]:
# score for White defendants
threshold  = 6
df_white = df[df['race']=="Caucasian"].copy()
df_white['is_med_or_high_risk'] = (df_white['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_white['two_year_recid'], df_white['is_med_or_high_risk'])
print("False positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))

False positive rate (White)      :  0.14717741935483872
False negative rate (White)      :  0.5921325051759835


In [11]:
# recitivism prediction for White defendants
threshold  = 6
df_white = df[df['race']=="Caucasian"].copy()
df_white['is_med_or_high_risk'] = (df_white['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_white['two_year_recid'], df_white['is_med_or_high_risk'])
print("Rate predicts recitivism (White)      : ", (tp +fn)/ (tn + fp + fn + tp))

Rate predicts recitivism (White)      :  0.39364303178484106


In [12]:
# recitivism prediction for Black defendants
threshold  = 6
df_white = df[df['race']=="African-American"].copy()
df_white['is_med_or_high_risk'] = (df_white['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_white['two_year_recid'], df_white['is_med_or_high_risk'])
print("Rate predicts recitivism (Black)      : ", (tp +fn)/ (tn + fp + fn + tp))

Rate predicts recitivism (Black)      :  0.5143398268398268


# our models

In [13]:
import pandas as pd
df = pd.read_csv('data/compas-scores-two-years.csv')
print(df.shape)
feats = ['race', 'sex', 'age_cat',  'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'two_year_recid']
df = df[ feats ]
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)
        
data_model  = pd.concat([
                df[ ['priors_count','two_year_recid'] ], 
                pd.get_dummies(df['race'], drop_first = True, prefix = 'race'),
                pd.get_dummies(df['sex'], drop_first = True, prefix = 'sex'),
                pd.get_dummies(df['age_cat'], drop_first = True, prefix = 'age_cat'),
                pd.get_dummies(df['juv_fel_count'], drop_first = True, prefix = 'juv_fel_count'),
                pd.get_dummies(df['juv_misd_count'], drop_first = True, prefix = 'juv_misd_count'),
                pd.get_dummies(df['juv_other_count'], drop_first = True, prefix = 'juv_other_count'),
                pd.get_dummies(df['c_charge_degree'], drop_first = True, prefix = 'c_charge_degree')
                ], axis = 1)
print(data_model.shape)


(7214, 53)
(6150, 9)
(6139, 9)
(6118, 9)
(6113, 9)
(6113, 18)


### Model with all features including 'race' (or unfair model)

In [14]:
## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)
clf_xgb_w_race = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators= 800, max_depth= 4, learning_rate= 0.05, seed = 42)
clf_xgb_w_race.fit(X_train.values, y_train, eval_metric="auc", verbose=False)

NameError: name 'xgb' is not defined

In [8]:
# score for Black defendants
X_test_ = X_test[ X_test['race_Caucasian']==0 ]
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ = clf_xgb_w_race.predict(X_test_.values)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))

False positive rate (Black)      :  0.3314285714285714
False negative rate (Black)      :  0.31754874651810583


In [9]:
# score for White defendants
X_test_ = X_test[ X_test['race_Caucasian']==1 ]
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ = clf_xgb_w_race.predict(X_test_.values)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))

False positive rate (White)      :  0.18360655737704917
False negative rate (White)      :  0.5741626794258373


### Model without 'race' (or unaware model)

In [10]:
## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)

cols = list(X_train.columns)
cols.remove('race_Caucasian')
X_train_wo_race = X_train[ cols ]
X_test_wo_race = X_test[ cols ]

clf_xgb_wo_race = xgb.XGBClassifier(objective = 'binary:logistic', n_estimators= 800, max_depth= 4, learning_rate= 0.05, seed = 42)
clf_xgb_wo_race.fit(X_train_wo_race.values, y_train, eval_metric="auc", verbose=False)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=800, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [11]:
# score for Black defendants
X_test_ = X_test_wo_race[ X_test['race_Caucasian']==0 ]
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ = clf_xgb_wo_race.predict(X_test_.values)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))

False positive rate (Black)      :  0.31142857142857144
False negative rate (Black)      :  0.3565459610027855


In [12]:
# score for White defendants
X_test_ = X_test_wo_race[ X_test['race_Caucasian']==1 ]
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ = clf_xgb_wo_race.predict(X_test_.values)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))

False positive rate (White)      :  0.21639344262295082
False negative rate (White)      :  0.5741626794258373


### Counter Fairness

In [13]:
# reference: https://github.com/fiorenza2/CFFair_Emulate

In [14]:
import pystan
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from argparse import ArgumentParser, ArgumentTypeError
from pathlib import Path
# wrapper class for statsmodels linear regression (more stable than SKLearn)
class SM_LinearRegression():
    def __init__(self):
        pass
        
    def fit(self, X, y):
        N = X.shape[0]
        self.LRFit = sm.OLS(y, np.hstack([X,np.ones(N).reshape(-1,1)]),hasconst=True).fit()
        
    def predict(self,X):
        N = X.shape[0]
        return self.LRFit.predict(np.hstack([X,np.ones(N).reshape(-1,1)]))

In [15]:
import pandas as pd
df = pd.read_csv('compas-scores-two-years.csv')
print(df.shape)
feats = ['race', 'sex', 'age_cat',  'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'two_year_recid']
df = df[ feats ]
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)
        
data_model  = pd.concat([
                df[ ['priors_count','two_year_recid'] ], 
                pd.get_dummies(df['race'], drop_first = True, prefix = 'race'),
                pd.get_dummies(df['sex'], drop_first = True, prefix = 'sex'),
                pd.get_dummies(df['age_cat'], drop_first = True, prefix = 'age_cat'),
                pd.get_dummies(df['juv_fel_count'], drop_first = True, prefix = 'juv_fel_count'),
                pd.get_dummies(df['juv_misd_count'], drop_first = True, prefix = 'juv_misd_count'),
                pd.get_dummies(df['juv_other_count'], drop_first = True, prefix = 'juv_other_count'),
                pd.get_dummies(df['c_charge_degree'], drop_first = True, prefix = 'c_charge_degree')
                ], axis = 1)
print(data_model.shape)

## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)


(7214, 53)
(6150, 9)
(6139, 9)
(6118, 9)
(6113, 9)
(6113, 18)


In [16]:
X_train.head()

,priors_count,race_Caucasian,sex_Male,age_cat_Greater than 45,age_cat_Less than 25,juv_fel_count_1,juv_fel_count_2,juv_fel_count_3,juv_fel_count_4,juv_misd_count_1,juv_misd_count_2,juv_misd_count_3,juv_other_count_1,juv_other_count_2,juv_other_count_3,juv_other_count_4,c_charge_degree_M
2232,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1009,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3056,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3717,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4702,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
linear_eps_priors_count = SM_LinearRegression()
linear_eps_priors_count.fit(np.vstack((X_train['race_Caucasian'].values.reshape(-1,1),X_test['race_Caucasian'].values.reshape(-1,1))),
               list(X_train['priors_count'])  + list(X_test['priors_count']) 
                           )
eps_priors_count_train = list(X_train['priors_count']) - linear_eps_priors_count.predict(X_train['race_Caucasian'].values.reshape(-1,1))
eps_priors_count_test = list(X_test['priors_count']) - linear_eps_priors_count.predict(X_test['race_Caucasian'].values.reshape(-1,1))


In [18]:
linear_eps_c_charge_degree_M = SM_LinearRegression()
linear_eps_c_charge_degree_M.fit(np.vstack((X_train['race_Caucasian'].values.reshape(-1,1),X_test['race_Caucasian'].values.reshape(-1,1))),
               list(X_train['c_charge_degree_M'])  + list(X_test['c_charge_degree_M']) 
                           )
eps_c_charge_degree_M_train = list(X_train['c_charge_degree_M']) - linear_eps_c_charge_degree_M.predict(X_train['race_Caucasian'].values.reshape(-1,1))
eps_c_charge_degree_M_test = list(X_test['c_charge_degree_M']) - linear_eps_c_charge_degree_M.predict(X_test['race_Caucasian'].values.reshape(-1,1))


In [19]:
# predict on target using abducted latents
smlr_L3 = SM_LinearRegression()
smlr_L3.fit(np.hstack((eps_priors_count_train.reshape(-1,1),eps_c_charge_degree_M_train.reshape(-1,1))),y_train)#.values.reshape(-1,1)

In [20]:
# predict on test epsilons
preds = smlr_L3.predict(np.hstack(( eps_priors_count_test.reshape(-1,1),eps_c_charge_degree_M_test.reshape(-1,1) )))
preds = [1 if e>.5 else 0 for e in preds]
preds = np.array(preds)

In [21]:
# score for Black defendants
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ =  preds[ X_test['race_Caucasian']==0 ]
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))

False positive rate (Black)      :  0.1657142857142857
False negative rate (Black)      :  0.5626740947075209


In [22]:
# score for White defendants
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ =  preds[ X_test['race_Caucasian']==1 ]
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))

False positive rate (White)      :  0.18688524590163935
False negative rate (White)      :  0.5933014354066986
